# Running an Inference on Custom Data

> **Note:** In this tutorial, we will be using a sample `RandLANet` `SemanticKITTI` weight file which we need to:
>
> 1. Download from: https://storage.googleapis.com/open3d-releases/model-zoo/randlanet_semantickitti_202201071330utc.pth
> 2. Place the downloaded `randlanet_semantickitti_202201071330utc.pth` file into `'Open3D-ML/docs/tutorial/notebook/'` subdirectory, or any other place and change the `ckpt_path` accordingly.
>
> For other model/dataset weight files, please check out https://github.com/isl-org/Open3D-ML#semantic-segmentation-1


An inference predicts the results based on the trained model. **Inference always depends on a prior step - _training_**.

> **Please see the [Training a Semantic Segmentation Model Using PyTorch](train_ss_model_using_pytorch.ipynb) and [Training a Semantic Segmentation Model Using TensorFlow](train_ss_model_using_tensorflow.ipynb) for training tutorials.**

To recap the model training process, - in our data model, we:

1. define a dataset;
2. create a training split part of the data;
3. define a model;
4. create a `SemanticSegmentation` `pipeline` object;
5. execute the `pipeline.run_train()` method which runs the training process.

While training, the model saves the checkpoint files every few epochs, in the *logs* directory. We use these trained weights to restore the model for inference.

Our first step in Inference on a Custom Data implementation is to import *Open3D-ML* and *Numpy* libraries: 


In [1]:
import open3d.ml.torch as ml3d  # just switch to open3d.ml.tf for tf usage
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.

--------------------------------------------------------------------------------

 Using the Open3D PyTorch ops with CUDA 11 may have stability issues!

 We recommend to compile PyTorch from source with compile flags
   '-Xcompiler -fno-gnu-unique'

 or use the PyTorch wheels at
   https://github.com/isl-org/open3d_downloads/releases/tag/torch1.8.2


 Ignore this message if PyTorch has been compiled with the aforementioned
 flags.

 See https://github.com/isl-org/Open3D/issues/3324 and
 https://github.com/pytorch/pytorch/issues/52663 for more information on this
 problem.

--------------------------------------------------------------------------------



2022-04-06 15:53:47.653694: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-06 15:53:47.653724: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


We then create a checkpoint path pointing to the data weights file we downloaded (generated at the end of  the Training stage):

In [2]:
# Download weights using link from model zoo (collection of weights for all combinations of model and dataset).
# Link : https://github.com/isl-org/Open3D-ML#semantic-segmentation-1
# Randlanet SemanticKITTI[PyTorch] : https://storage.googleapis.com/open3d-releases/model-zoo/randlanet_semantickitti_202201071330utc.pth
ckpt_path = './randlanet_semantickitti_202201071330utc.pth'

Now, we define a `dataset` object identical to how it was done in our previous *Training a Semantic Segmentation Model* tutorials:

In [3]:
# We define dataset (similar to train_ss_using_pytorch tutorial)
dataset = ml3d.datasets.SemanticKITTI(dataset_path='SemanticKITTI/', cache_dir='./logs/cache',training_split=['00'], validation_split=['01'], test_split=['01'])


Then, initialize `model` and `pipeline` objects:

In [4]:
# Initializing the model and pipeline
model = ml3d.models.RandLANet(in_channels=3)
pipeline = ml3d.pipelines.SemanticSegmentation(model)

Next, we restore the model with our data weights file with `pipeline.load_ckpt()` method:

In [5]:
# Load checkpoint using `load_ckpt` method (restoring weights for inference)
pipeline.load_ckpt(ckpt_path=ckpt_path)

INFO - 2022-04-06 15:54:37,266 - semantic_segmentation - Loading checkpoint ./randlanet_semantickitti_202201071330utc.pth


Now, lets query the first pointcloud from the `test` split.

In [6]:
test_data = dataset.get_split('test')
data = test_data.get_data(0)

INFO - 2022-04-06 15:54:40,026 - semantickitti - Found 1 pointclouds for test


Let's display the content `data` contains:

In [7]:
print(data)

{'point': array([[ 5.2305943e+01,  2.2989707e-02,  1.9779946e+00],
       [ 5.3259735e+01,  1.0695236e-01,  2.0099745e+00],
       [ 5.3284321e+01,  2.7487758e-01,  2.0109341e+00],
       ...,
       [ 3.8249431e+00, -1.4261885e+00, -1.7655631e+00],
       [ 3.8495324e+00, -1.4222100e+00, -1.7755738e+00],
       [ 3.8631279e+00, -1.4142324e+00, -1.7805853e+00]], dtype=float32), 'feat': None, 'label': array([0, 0, 0, ..., 9, 9, 9], dtype=int32)}


For inference on custom data, you can convert your pointcloud into this format:

**Dictionary with keys {'point', 'feat', 'label'}**

If you already have the *ground truth labels*, you can add them to data to get accuracy and IoU (Intersection over Union). Otherwise, pass labels as `None`.

And now - the main topic of our tutorial - running an inference on the test data. You can call the `run_inference()` method with your data, - it will print accuracy per class and Intersection over Union (IoU) metrics. The last entry in the list is *mean accuracy* and *mean IoU*:

In [9]:
# Running inference on test data
results = pipeline.run_inference(data)
# prints per class accuracy and IoU (Intersection over Union). Last entry is mean accuracy and mean IoU.
# We get several `nan` outputs for missing classes in the input data.


test 0/1: 100%|█████████████████████████| 83500/83500 [01:14<00:00, 1115.74it/s]

test 0/1: 100%|████████████████████████▉| 83262/83500 [00:09<00:00, 9105.02it/s]INFO - 2022-04-06 15:57:33,626 - semantic_segmentation - Accuracy : [0.9721905630271256, nan, nan, nan, nan, nan, nan, 0.0, 0.9554558517345533, 0.9201252936570086, 0.47223502304147463, nan, 0.9598018769551616, 0.9974093264248705, 0.8371072429561882, 0.8510466988727858, 0.9349005424954792, 0.9367521367521368, 0.67, 0.7922520463263987]
INFO - 2022-04-06 15:57:33,627 - semantic_segmentation - IoU : [0.9032189750105888, nan, 0.0, nan, 0.0, nan, nan, 0.0, 0.9118182072593961, 0.21114106019766396, 0.464405755504362, 0.0, 0.9425046078230596, 0.11663132384126022, 0.8259888650340235, 0.753922967189729, 0.6790123456790124, 0.8404907975460123, 0.5775862068965517, 0.481781407465444]

test 0/1: 100%|█████████████████████████| 83500/83500 [00:25<00:00, 9105.02it/s]

The `results` object will return dictionary of predicted labels and predicted probabilities per point:

In [10]:
# Dictionary of predicted labels and predicted probabilities per class
results

{'predict_labels': array([12, 12, 12, ...,  8,  8,  8], dtype=int16),
 'predict_scores': array([[2.044e-05, 1.186e-05, 8.345e-07, ..., 1.608e-04, 2.861e-03,
         1.152e-03],
        [1.615e-05, 1.079e-05, 8.345e-07, ..., 2.084e-04, 5.005e-03,
         2.203e-03],
        [1.317e-05, 9.954e-06, 6.557e-07, ..., 1.574e-04, 6.054e-03,
         2.481e-03],
        ...,
        [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [5.960e-08, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [5.960e-08, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
         0.000e+00]], dtype=float16)}